In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder,FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
from imblearn.over_sampling import SMOTE
from collections import Counter
import pandas as pd
import torch
from torch import nn
from torch import optim 

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.nominal_data = ['Gender','Working Professional or Student','Profession','Degree','Sleep Duration','Dietary Habits']
        self.allowed_profession = ['Teacher','Content Writer','Architect','Consultant','HR Manager','Pharmacist','Doctor','Business Analyst','Entrepreneur','Chemist']
        self.allowed_sleepduration = ['Less than 5 hours','7-8 hours','More than 8 hours','5-6 hours','3-4 hours']
        self.allowed_habits = ['Moderate','Unhealthy','Healthy']
        self.allowed_degree = ['Class 12','B.Ed','B.Arch','B.Com','B.Pharm','BCA','M.Ed','MCA','BBA','BSc']
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        df.drop(['id','Name','City'], axis=1, inplace=True)

        # Set academic values to 0 for professionals
        df.loc[df['Working Professional or Student'] == 'Working Professional', ['CGPA', 'Academic Pressure', 'Study Satisfaction']] = 0.0

        # Fill missing values
        for col in ['Profession', 'Degree', 'Dietary Habits']:
            df[col] = df[col].fillna(df[col].mode()[0])
        for col in ['Work Pressure', 'Academic Pressure', 'Job Satisfaction', 'Financial Stress', 'CGPA', 'Study Satisfaction']:
            df[col] = df[col].fillna(df[col].median())

        # Map uncommon categories to 'Others'
        df['Profession'] = df['Profession'].where(df['Profession'].isin(self.allowed_profession), 'Others')
        df['Sleep Duration'] = df['Sleep Duration'].where(df['Sleep Duration'].isin(self.allowed_sleepduration), 'Others')
        df['Dietary Habits'] = df['Dietary Habits'].where(df['Dietary Habits'].isin(self.allowed_habits), 'Others')
        df['Degree'] = df['Degree'].where(df['Degree'].isin(self.allowed_degree), 'Others')

        # Encode ordinal
        df['Family History of Mental Illness'] = df['Family History of Mental Illness'].replace({'No': 0, 'Yes': 1})
        df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].replace({'No': 0, 'Yes': 1})

        # One-hot encoding
        df = pd.get_dummies(df, columns=self.nominal_data, dtype='int')

        # Handle outliers
        for col in ['Academic Pressure','CGPA','Study Satisfaction']:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            df[col] = np.clip(df[col], lower, upper)

        return df


# using custom class

In [4]:

preprocessing_pipeline = Pipeline([
    ('custom_preprocessor', CustomPreprocessor())
])


In [5]:
data=preprocessing_pipeline.fit_transform(X)

C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7284\420157920.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Family History of Mental Illness'] = df['Family History of Mental Illness'].replace({'No':0,'Yes':1})
C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7284\420157920.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].replace({'No':0,'Yes':1})


In [ ]:
class DNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(DNN,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[0],hidden_size[1]),
            nn.ReLU(),
            nn.Linear(hidden_size[1],hidden_size[2]),
            nn.ReLU(),
            nn.Linear(hidden_size[2],hidden_size[3]),
            nn.ReLU(),
            nn.Linear(hidden_size[3],output_size),
            nn.Sigmoid()
        )
        
    def forward(self,X):
        X = self.layers(X)
        return X
model = DNN(input_size,hidden_size,output_size)
criterion= nn.BCELoss()
# optimizer = optim.SGD(model.parameters())
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Step 2: Read and split data
df = pd.read_csv(r"C:\Users\MY Laptop\Desktop\guvi_class\mental health survey\train.csv")
X = df.drop(['Depression'],axis=1)
y = df['Depression']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Step 3: Preprocess
# preprocessor = Pipeline([('preprocessor', FunctionTransformer(preprocessor))])
X_train = preprocessing_pipeline.fit_transform(X_train)
X_test = preprocessing_pipeline.transform(X_test)

Step 4: SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Step 5: Convert to torch tensors
X_train_tensor = torch.tensor(X_train_res.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_res.values.reshape(-1,1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.reshape(-1,1), dtype=torch.float32)

# Step 6: Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Step 7: Define your DNN model
class DNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.ReLU(),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.ReLU(),
            nn.Linear(hidden_size[2], hidden_size[3]),
            nn.ReLU(),
            nn.Linear(hidden_size[3], output_size),
            nn.Sigmoid()
        )
        
    def forward(self, X):
        return self.layers(X)

input_size = X_train_tensor.shape[1]
hidden_size = [32,16,8,4]
output_size = 1

model = DNN(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Step 8: Train the model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# Step 9: Evaluate on test set
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor)
    y_pred_labels = (y_pred_test >= 0.5).float()

from sklearn.metrics import classification_report
print(classification_report(y_test_tensor.numpy(), y_pred_labels.numpy()))


C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7284\4100365638.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Family History of Mental Illness'] = df['Family History of Mental Illness'].replace({'No': 0, 'Yes': 1})
C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7284\4100365638.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].replace({'No': 0, 'Yes': 1})
C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7284\4100365638.py:36: Fut

Epoch 1/30, Loss: 0.1575
Epoch 2/30, Loss: 0.2650
Epoch 3/30, Loss: 0.2694
Epoch 4/30, Loss: 0.1945
Epoch 5/30, Loss: 0.0699
Epoch 6/30, Loss: 0.0428
Epoch 7/30, Loss: 0.1740
Epoch 8/30, Loss: 0.5218
Epoch 9/30, Loss: 0.1501
Epoch 10/30, Loss: 0.0620
Epoch 11/30, Loss: 0.0419
Epoch 12/30, Loss: 0.0884
Epoch 13/30, Loss: 0.1851
Epoch 14/30, Loss: 0.0939
Epoch 15/30, Loss: 0.1002
Epoch 16/30, Loss: 0.1528
Epoch 17/30, Loss: 0.0688
Epoch 18/30, Loss: 0.0251
Epoch 19/30, Loss: 0.5054
Epoch 20/30, Loss: 0.1155
Epoch 21/30, Loss: 0.0713
Epoch 22/30, Loss: 0.0324
Epoch 23/30, Loss: 0.0507
Epoch 24/30, Loss: 0.4353
Epoch 25/30, Loss: 0.1218
Epoch 26/30, Loss: 0.1346
Epoch 27/30, Loss: 0.0196
Epoch 28/30, Loss: 0.2657
Epoch 29/30, Loss: 0.1036
Epoch 30/30, Loss: 0.0868
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.95     23027
         1.0       0.82      0.75      0.78      5113

    accuracy                           0.93     28140
   macro av

In [10]:
ohe_columns
joblib.dump(ohe_columns, 'ohe_columns.pkl')

['ohe_columns.pkl']

In [11]:
torch.save(model.state_dict(), 'mental_health_dnn.pth')
# To load later:
# model.load_state_dict(torch.load('mental_health_dnn.pth'))
# model.eval()


In [16]:
import joblib
joblib.dump(preprocessing_pipeline, 'preprocessor.pkl')


['preprocessor.pkl']